## Getting graph metrics using the GNM Toolbox

In [1]:
import torch; print(torch.backends.mps.is_available())

True


In [2]:
# export OMP_NUM_THREADS=$(sysctl -n hw.perflevel0.physicalcpu 2>/dev/null || sysctl -n hw.ncpu)
# export OPENBLAS_NUM_THREADS=$OMP_NUM_THREADS
# export VECLIB_MAXIMUM_THREADS=$OMP_NUM_THREADS
# export NUMEXPR_NUM_THREADS=$OMP_NUM_THREADS


In [3]:
import os
notebook_dir = os.getcwd() 

import sys
loc = os.path.abspath(os.path.join(notebook_dir, '..', '..', 'src'))
sys.path.append(loc)

import time
import torch
from gnm import *
from gnm import defaults

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# binary_consensus_network = defaults.get_binary_network(device=DEVICE)

# distance_matrix = torch.load("/Users/adrian/Documents/01_projects/14_4D_lab/src/imported_libraries/GenerativeNetworkModels_2/src/gnm/defaults/distance_matrices/AAL_DISTANCES.pt")
binary_consensus_network = torch.load("/Users/adrian/Documents/01_projects/14_4D_lab/src/imported_libraries/GenerativeNetworkModels_2/src/gnm/defaults/binary_networks/CALM_BINARY_CONSENSUS.pt")
# weighted_consensus_network = torch.load("/Users/adrian/Documents/01_projects/14_4D_lab/src/imported_libraries/GenerativeNetworkModels_2/src/gnm/defaults/weighted_networks/CALM_WEIGHTED_CONSENSUS.pt")


Here, we show the benefit of using GNM Toolbox over alternative BCT package for calculating betweenness centrality

In [16]:
from gnm.utils import binary_betweenness_centrality, binary_clustering_coefficients, binary_characteristic_path_length
import bct
import time
import networkx as nx

# create 100 iterations of the same network
repeated_network = torch.repeat_interleave(binary_consensus_network, 100, dim=0)
repeated_network_np = repeated_network.detach().cpu().numpy()
print("Repeated network shape: ", repeated_network.shape)

# calculate betweenness centrality using gnm
start = time.time()
betweenness = binary_betweenness_centrality(repeated_network)
end = time.time()
print("Betweenness GNM time: ", end - start)

# calculate betweenness centrality using bct
start = time.time()
for i in range(repeated_network_np.shape[0]):
    network = repeated_network_np[i:i+1].reshape(repeated_network.shape[1], repeated_network.shape[2])
    betweenness = bct.betweenness_bin(network)
end = time.time()
print("Betweenness BCT time: ", end - start)


Repeated network shape:  torch.Size([100, 90, 90])
Betweenness GNM time:  0.8877272605895996
Betweenness BCT time:  0.07578492164611816


In [ ]:
# import torch
# import gnm 
# # from gnm import get_device
# # device = get_device()  # mps if available&desired, else cuda, else cpu

# from gnm import defaults


# # Keep everything on device, convert only when calling BCT/NumPy:
# repeated_network = torch.repeat_interleave(binary_consensus_network, 100, dim=0)
# # For BCT (NumPy), convert at the edge:
# repeated_network_np = repeated_network.detach().cpu().numpy()


# for name, dev in [("cpu", torch.device("cpu")),
#                   ("mps", torch.device("mps")) if torch.backends.mps.is_available() else ()]:
#     if not dev: continue
#     A = repeated_network.to(dev)
#     t0 = time.time(); _ = binary_betweenness_centrality(A); t1 = time.time()
#     print(name, t1 - t0)

cpu 0.8639421463012695
mps 4.828585147857666
